# One BLOCK at a time

## Recap

Our initial message:

In [1]:
message = "Hi!"
message

'Hi!'

We have our message converted from text to ascii to binary:

In [2]:
binary_message = ''.join(format(ord(char), '08b') for char in message)
binary_message

'010010000110100100100001'

We have our padded message.

In [3]:
def pad_binary(binary_message,
               block_size=512,
               length_field=64):

    # Add the '1' bit
    padded = binary_message + '1'

    # Calculate Zeros Needed
    msg_size_on_final_block = (len(padded) % block_size)
    zeros_needed = block_size - msg_size_on_final_block - length_field

    # If there are not enough zeros, add another block w/ zeros
    if zeros_needed < 0:
        zeros_needed += 512
    
    # Otherwise, append the zeros needed
    padded += '0' * zeros_needed

    # Add 64-bit message length to the end, padded with zeros
    msg_length = len(binary_message)
    length_bits = format(msg_length, '064b')
    final_padded = padded + length_bits

    return final_padded

pad_binary(binary_message)

'01001000011010010010000110000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000011000'

## Messages Longer than One Block

In [4]:
message = "This is a longer message that will require more than one 512-bit block, so we will need a function to break them down."
binary_message = ''.join(format(ord(char), '08b') for char in message)
padded_message = pad_binary(binary_message)

The padded message is too long. Needs to be broken down into 512-bit blocks.

In [5]:
def divide_into_blocks(binary_string, block_size=512):
    """
    Divides a binary string into blocks of a specified size.

    Args:
        binary_string (str): The binary string to be divided.
        block_size (int): The size of each block in bits (default is 512).

    Returns:
        list: A list of binary string blocks.
    """
    # Ensure the binary string length is a multiple of 8
    if len(binary_string) % 8 != 0:
        raise ValueError("Binary string length must be a multiple of 8")

    # Split the binary string into blocks
    blocks = []
    for i in range(0, len(binary_string), block_size):
        blocks.append(binary_string[i:i+block_size])
    
    return blocks

And here we can see the blocks in both binary format.

In [6]:
# Divide the padded binary string into 512-bit blocks
blocks = divide_into_blocks(padded_message)

# Print the blocks in binary format
print("Blocks in Binary Format:")
for i, block in enumerate(blocks):
    print(f"Block {i + 1}: {block}")


Blocks in Binary Format:
Block 1: 01010100011010000110100101110011001000000110100101110011001000000110000100100000011011000110111101101110011001110110010101110010001000000110110101100101011100110111001101100001011001110110010100100000011101000110100001100001011101000010000001110111011010010110110001101100001000000111001001100101011100010111010101101001011100100110010100100000011011010110111101110010011001010010000001110100011010000110000101101110001000000110111101101110011001010010000000110101001100010011001000101101011000100110100101110100
Block 2: 001000000110001001101100011011110110001101101011001011000010000001110011011011110010000001110111011001010010000001110111011010010110110001101100001000000110111001100101011001010110010000100000011000010010000001100110011101010110111001100011011101000110100101101111011011100010000001110100011011110010000001100010011100100110010101100001011010110010000001110100011010000110010101101101001000000110010001101111011101110110111000101110100000000000

But we can also see it in byte string format.

In [7]:
# Convert binary blocks to byte strings and print
print("\nBlocks in Byte String Format:")
for i, block in enumerate(blocks):
    byte_array = bytearray()
    for j in range(0, len(block), 8):
        byte = block[j:j+8]
        byte_array.append(int(byte, 2))
    byte_string = bytes(byte_array)
    print(f"Block {i + 1}: {byte_string}")


Blocks in Byte String Format:
Block 1: b'This is a longer message that will require more than one 512-bit'
Block 2: b' block, so we will need a function to break them down.\x80\x00\x00\x00\x00\x00\x00\x00\x03\xb0'


#### What's bytestring format?

Reasons to Use Byte Strings
Standard Library Functions:

- Many standard library functions and cryptographic operations expect data in byte string format. For example, the struct.unpack function used in the prepare_message_schedule function requires byte strings.
Efficiency:

- Byte strings are more memory-efficient and faster to process compared to binary strings. Each byte in a byte string represents 8 bits, whereas a binary string uses one character per bit.
Compatibility:

- Byte strings are the standard way to handle binary data in Python and other programming languages. This ensures compatibility with various libraries and tools.

`b''`

A **byte literal** in Python is a sequence of bytes, represented by a string prefixed with b. It is used to handle binary data directly.

In [27]:
# Creating a byte literal
byte_literal = b'Hello, World!'

# Printing the byte literal
print(byte_literal)  # Output: b'Hello, World!'

# Accessing individual bytes
print(byte_literal[0])  # Output: 72 (ASCII value of 'H')
print(byte_literal[1])  # Output: 101 (ASCII value of 'e')

# Iterating over the byte literal
for byte in byte_literal:
    print(byte, end=' ')  # Output: 72 101 108 108 111 44 32 87 111 114 108 100 33

b'Hello, World!'
72
101
72 101 108 108 111 44 32 87 111 114 108 100 33 

`bytes()`
- `b''`: Use for static, predefined byte data.
- `bytes()`: Use for dynamic creation and conversion of byte data from various sources.

**Empty Byte Object:**

In [28]:
empty_bytes = bytes()
print(empty_bytes)  # Output: b''

b''


**From an Iterable of Integers:**

In [29]:
byte_obj = bytes([72, 101, 108, 108, 111])
print(byte_obj)  # Output: b'Hello'

b'Hello'


**From a String with Encoding:**

In [30]:
byte_obj = bytes("Hello", "utf-8")
print(byte_obj)  # Output: b'Hello'

b'Hello'


**From an Existing Byte Object:**

In [31]:
original = b'Hello'
copy = bytes(original)
print(copy)  # Output: b'Hello'

b'Hello'


### Exercise: Create a Byte Literal:

1. Create a byte literal representing the string "Python is fun!".

In [36]:
print(b'Python is fun!')

b'Python is fun!'


2. Convert the list [80, 121, 116, 104, 111, 110] to a byte object using the bytes() function.

In [35]:
bytes_object = bytes([80, 121, 116, 104, 111, 110])
print(bytes_object)

b'Python'


3. Convert the string `"Hello, World!"` to a byte object using UTF-8 encoding with the `bytes()` function.

In [46]:
# 3. Convert a String to Bytes with Encoding
bytes_object = bytes("Hello, World!", "utf-8")
print(bytes_object)  # Output: b'Hello, World!'

[byte for byte in bytes_object]

b'Hello, World!'


[72, 101, 108, 108, 111, 44, 32, 87, 111, 114, 108, 100, 33]

In [54]:
# Create a byte object from a string using UTF-8 encoding
bytes_object = bytes("Hello, World!", "utf-8")
print(bytes_object)  # Output: b'Hello, World!'

# Print individual elements in the byte object
for byte in bytes_object:
    print(byte, end=' ')  # Output: 72 101 108 108 111 44 32 87 111 114 108 100 33

b'Hello, World!'
72 101 108 108 111 44 32 87 111 114 108 100 33 

In [48]:
# Decode the byte object back to a string
decoded_string = bytes_object.decode("utf-8")
print(decoded_string)  # Output: Hello, World!

[byte for byte in decoded_string]

Hello, World!


['H', 'e', 'l', 'l', 'o', ',', ' ', 'W', 'o', 'r', 'l', 'd', '!']

### `struct`

Sure, let's dive deeper into the struct module in Python and its usage.

What is the struct Module?

The struct module in Python provides functions to convert between Python values and C structs represented as Python bytes objects. This is useful for handling binary data stored in files or coming from network connections, among other things.

`struct.pack(format, v1, v2, ...)`

Packs the given values into a bytes object according to the specified format.

Yes, that's a good summary of what `struct.pack` does. Let's break it down step-by-step:

### Summary of `struct.pack`

1. **Purpose**:
   - Converts Python values (e.g., integers, floats, strings) into a binary format (byte object).

2. **Format String**:
   - Specifies the data types and the order in which the values should be packed.
   - Indicates endianness (byte order) and data types.

3. **Endianness**:
   - **`>`**: Big-endian (most significant byte first).
   - **`<`**: Little-endian (least significant byte first).

4. **Data Types**:
   - **`I`**: Unsigned 32-bit integer.
   - **`h`**: Signed 16-bit integer.
   - **`c`**: Single byte character.
   - **`f`**: 32-bit floating point.
   - And many more...

In [81]:
import struct

# Define the values to be packed
integer_value = 1214849408  # Unsigned 32-bit integer
short_value = 32000         # Signed 16-bit integer
char_value = b'A'           # Single byte character
float_value = 3.14          # 32-bit floating point

# Define the format string
format_string = '>Ihcf' # >: Big-endian, store bytes in normal order
                        # I: Unsigned 32-bit int
                        # h: Signed 16-bit int
                        # c: Char
                        # f: Float

# Pack the values into a bytes object
packed_data = struct.pack(format_string, integer_value, short_value, char_value, float_value)

# Print the packed data
print(f"Packed data: {packed_data}")
print("Packed data (hex):", ' '.join(f'0x{byte:02x}' for byte in packed_data))

Packed data: b'Hi!\x80}\x00A@H\xf5\xc3'
Packed data (hex): 0x48 0x69 0x21 0x80 0x7d 0x00 0x41 0x40 0x48 0xf5 0xc3


`struct.unpack()`

In [82]:
# Unpack the data back into Python values
unpacked_data = struct.unpack(format_string, packed_data)

# Print the unpacked data
print(f"Unpacked data: {unpacked_data}")

Unpacked data: (1214849408, 32000, b'A', 3.140000104904175)
